In [1]:
import numpy as np
from BPmodule.BPGen import PROBLEM2, PROBLEM3, PROBLEM4, PROBLEM5
from BPnumba.GeneticOperators import CreateHeuristicPob,Ind
from BPnumba.BPPdat import DBLF,DBLF2,create_Bin
from numba import njit, objmode, prange
import time
from numba.typed import List as NumbaList
from typing import List
from BPmodule.DataLecture import ExpotarJSON,PossiblePositions

In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]

#PATHBOX = "C:/Users/nicoo/BPP/Assets/bpp.json"
#PATHPOINT = "C:/Users/nicoo/BPP/Assets/points.json"

In [27]:
contenedor = P5A1[0][0]
datos = P5A1[1]
n = len(datos)
sameInstance = CreateHeuristicPob(50,datos,True)
n

155

In [28]:
suma = 0
for i in np.arange(n):
    suma += np.prod(datos[i])
suma-np.prod(contenedor)

0

In [29]:
ind = Ind(NumbaList(sameInstance[1]))
ind.genome

ListType[int64]([4, 3, 19, 20, 21, 8, 9, 86, 84, 87, 85, 51, 53, 50, 52, 55, 57, 54, 56, 94, 92, 93, 91, 90, 89, 88, 6, 5, 7, 131, 129, 130, 128, 82, 83, 81, 38, 37, 36, 109, 110, 111, 119, 118, 117, 116, 16, 17, 18, 1, 2, 42, 41, 40, 39, 67, 69, 71, 66, 68, 70, 72, 14, 15, 95, 97, 96, 98, 76, 74, 75, 73, 27, 127, 125, 26, 126, 124, 28, 23, 139, 138, 137, 136, 25, 22, 24, 135, 134, 133, 34, 132, 33, 35, 30, 32, 48, 60, 29, 112, 58, 47, 49, 31, 114, 61, 59, 113, 115, 99, 101, 100, 102, 64, 103, 62, 105, 104, 65, 63, 13, 11, 12, 10, 107, 106, 149, 108, 148, 151, 150, 80, 78, 79, 44, 46, 77, 43, 45, 145, 147, 144, 146, 123, 122, 121, 120, 153, 152, 141, 155, 143, 154, 140, 142, ...])

In [30]:
bin = create_Bin(NumbaList(contenedor))
DBLF2(bin,ind.genome,datos)
print(bin.getLoadVol()/np.prod(contenedor))
print(bin.getN(),bin.getBoxes())

0.9273746666666667
145 [4, 3, 19, 20, 21, 8, 86, 93, 91, 89, 87, 88, 111, 119, 118, 117, 116, 33, 35, 29, 84, 85, 130, 66, 68, 83, 70, 72, 31, 95, 97, 96, 98, 73, 113, 122, 115, 120, 100, 144, 102, 81, 146, 123, 128, 28, 121, 24, 140, 67, 142, 69, 71, 126, 37, 47, 49, 50, 149, 52, 54, 124, 56, 129, 148, 82, 151, 150, 153, 61, 51, 110, 18, 53, 152, 155, 154, 43, 45, 59, 99, 101, 104, 65, 63, 105, 147, 145, 108, 143, 141, 114, 41, 106, 39, 103, 77, 107, 75, 109, 48, 112, 44, 26, 46, 38, 22, 79, 36, 55, 57, 131, 40, 58, 62, 42, 127, 17, 125, 90, 7, 74, 32, 60, 64, 78, 16, 2, 76, 138, 136, 1, 14, 15, 34, 30, 139, 11, 134, 132, 13, 12, 135, 80, 10, ...]


In [31]:
bin = create_Bin(NumbaList(contenedor))
DBLF(bin,ind.genome,datos)
print(bin.getLoadVol()/np.prod(contenedor))
print(bin.getN(),bin.getBoxes())

0.9414875185185185
142 [4, 3, 19, 20, 21, 8, 9, 86, 84, 87, 85, 51, 53, 50, 52, 55, 57, 54, 56, 93, 91, 90, 89, 88, 7, 131, 129, 130, 128, 82, 83, 81, 38, 37, 36, 109, 110, 111, 119, 118, 117, 116, 16, 17, 18, 1, 2, 42, 41, 40, 39, 67, 69, 71, 66, 68, 70, 72, 14, 15, 95, 97, 96, 98, 76, 74, 75, 73, 27, 127, 125, 26, 126, 124, 28, 23, 139, 137, 25, 22, 24, 34, 33, 35, 32, 48, 60, 29, 112, 58, 47, 49, 31, 114, 61, 59, 113, 115, 99, 101, 100, 102, 64, 103, 62, 105, 104, 65, 63, 13, 12, 107, 106, 149, 108, 148, 151, 150, 80, 78, 79, 44, 46, 77, 43, 45, 145, 147, 144, 146, 123, 122, 121, 120, 153, 152, 141, 155, 143, 154, 140, 142, ...]


In [32]:

#ExpotarJSON(PATHBOX,bin.getPositions(),bin.getBoxes(),datos,contenedor)
#PossiblePositions(PATHPOINT,bin.getPositions())